In [2]:
import pandas as pd
import numpy as np
import os
import sys
import json
import requests
import gc
from tqdm import tqdm
from datetime import datetime, timedelta
pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


In [3]:
access_key = pd.read_csv('../../cp_data.csv').iloc[0,1]
secret_key = pd.read_csv('../../cp_data.csv').iloc[1,1]

Order

5/9 저녁 기준, 5/3일 초과의 취소 데이터는 없었음. *요청 거부

In [477]:
import requests 
import hmac
import hashlib
from time import gmtime, strftime

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = "/v2/providers/affiliate_open_api/apis/openapi/v1/reports/orders"
# URL = "/v2/providers/affiliate_open_api/apis/openapi/v1/reports/cancels"

# Replace with your own ACCESS_KEY, SECRET_KEY, startDate, endDate
ACCESS_KEY = access_key
SECRET_KEY = secret_key
startDate = "20240430"
endDate = "20240430"
# Assuming subId is not required, if needed add it to the query parameters

def generateHmac(method, url, secretKey, accessKey, queryString=""):
    # Correct formatting for date-time if needed
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString

    signature = hmac.new(bytes(secretKey, 'utf-8'), 
                         message.encode('utf-8'), 
                         hashlib.sha256).hexdigest()

    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetimeGMT, signature)


def fetch_data(page):
    queryString = "startDate={}&endDate={}&page={}".format(startDate, endDate, page)
    fullURL = "{}{}?{}".format(DOMAIN, URL, queryString)
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    response = requests.request(method=REQUEST_METHOD, url=fullURL,
                                headers={
                                    "Authorization": authorization,
                                    "Content-Type": "application/json"
                                })
    return response.json()

def main():
    page = 0
    results = []

    while True:
        try:
            response = fetch_data(page)
            if 'data' in response:
                results.extend(response['data'])
                if len(response['data']) < 1000:
                    break
                page += 1
                print(response['data'])
            else:
                # Handle errors or missing 'data' key
                print(f"Error retrieving data: {response.get('message', 'No error message available')}")
                break
        except:
            print("pass")
        print(f"Total items fetched: {len(results)}")

    return results

# Execute the main function to handle pagination
all_data = main()
df = pd.DataFrame(all_data)

[{'date': '20240430', 'trackingCode': 'AF3155932', 'subId': 'homecoin', 'subParam': '73580666-071c-48b6-8a11-362b564eff8b_20240430190631', 'addtag': '460', 'ctag': '7072823434', 'orderId': 8684942817617311, 'productId': 7440025812, 'productName': '[리우] 레이스 브라 팬티 8종 세트', 'quantity': 1, 'gmv': 34800.0, 'commissionRate': 5.25, 'commission': 1827.0}, {'date': '20240430', 'trackingCode': 'AF3155932', 'subId': 'homecoin', 'subParam': 'dc59d1ec-bc33-45ff-893e-ab6dfbac8216_20240429185859', 'addtag': '400', 'ctag': '7978676284', 'orderId': 8364941617637919, 'productId': 7183750807, 'productName': '매일바이오 드링킹 플레인 요구르트', 'quantity': 3, 'gmv': 9400.0, 'commissionRate': 5.25, 'commission': 494.0}, {'date': '20240430', 'trackingCode': 'AF3155932', 'subId': 'homecoin', 'subParam': '3c88cfba-5fd2-48c4-9de5-ef1dafc978b0_20240429142342', 'addtag': '460', 'ctag': '7941549408', 'orderId': 8864943117968240, 'productId': 237181694, 'productName': '샤바스 디자인이 예쁜 샤워타월 스퀘어', 'quantity': 1, 'gmv': 8370.0, 'commiss

4/30일데이터 기준, 0430 ~ 0430 조회와 0403 ~ 0501 조회 데이터의 행 수가 다르다. 쿠팡 개병신인가?

In [478]:
len(df[df['date']=="20240430"])

1620

In [424]:
len(df[df['date']=="20240430"])

1620

In [426]:
len(df[df['date']=="20240430"])

1620

In [445]:
df.columns

Index(['date', 'trackingCode', 'subId', 'subParam', 'addtag', 'ctag',
       'orderId', 'productId', 'productName', 'quantity', 'gmv',
       'commissionRate', 'commission'],
      dtype='object')

In [479]:
len(api[api['orderDate']=="20240430"])
api_ = api[api['orderDate']=="20240430"]
api_.rename(columns={'orderDate':'date'},inplace=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_4412\654439807.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  api_.rename(columns={'orderDate':'date'},inplace=True)


In [455]:
df[df['orderId']==8234941917858815]

,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission


In [474]:
import requests 
import hmac
import hashlib
from time import gmtime, strftime

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = "/v2/providers/affiliate_open_api/apis/openapi/v1/reports/orders"
# URL = "/v2/providers/affiliate_open_api/apis/openapi/v1/reports/cancels"

# Replace with your own ACCESS_KEY, SECRET_KEY, startDate, endDate
ACCESS_KEY = access_key
SECRET_KEY = secret_key
startDate = "20240430"
endDate = "20240430"
# Assuming subId is not required, if needed add it to the query parameters

def generateHmac(method, url, secretKey, accessKey, queryString=""):
    # Correct formatting for date-time if needed
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString

    signature = hmac.new(bytes(secretKey, 'utf-8'), 
                         message.encode('utf-8'), 
                         hashlib.sha256).hexdigest()

    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetimeGMT, signature)

def fetch_data(page):
    queryString = "startDate={}&endDate={}&page={}".format(startDate, endDate, page)
    fullURL = "{}{}?{}".format(DOMAIN, URL, queryString)
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    response = requests.request(method=REQUEST_METHOD, url=fullURL,
                                headers={
                                    "Authorization": authorization,
                                    "Content-Type": "application/json"
                                })
    return response.json()

def main():
    page = 0
    results = []

    while True:
        try:
            response = fetch_data(page)
            if 'data' in response:
                results.extend(response['data'])
                if len(response['data']) < 1000:
                    break
                page += 1
                print(response['data'])
            else:
                # Handle errors or missing 'data' key
                print(f"Error retrieving data: {response.get('message', 'No error message available')}")
                break
        except:
            print("pass")
        print(f"Total items fetched: {len(results)}")

    return results


# Execute the main function to handle pagination
all_data = main()
df = pd.DataFrame(all_data)

[{'date': '20240430', 'trackingCode': 'AF3155932', 'subId': 'homecoin', 'subParam': '73580666-071c-48b6-8a11-362b564eff8b_20240430190631', 'addtag': '460', 'ctag': '7072823434', 'orderId': 8684942817617311, 'productId': 7440025812, 'productName': '[리우] 레이스 브라 팬티 8종 세트', 'quantity': 1, 'gmv': 34800.0, 'commissionRate': 5.25, 'commission': 1827.0}, {'date': '20240430', 'trackingCode': 'AF3155932', 'subId': 'homecoin', 'subParam': 'dc59d1ec-bc33-45ff-893e-ab6dfbac8216_20240429185859', 'addtag': '400', 'ctag': '7978676284', 'orderId': 8364941617637919, 'productId': 7183750807, 'productName': '매일바이오 드링킹 플레인 요구르트', 'quantity': 3, 'gmv': 9400.0, 'commissionRate': 5.25, 'commission': 494.0}, {'date': '20240430', 'trackingCode': 'AF3155932', 'subId': 'homecoin', 'subParam': '3c88cfba-5fd2-48c4-9de5-ef1dafc978b0_20240429142342', 'addtag': '460', 'ctag': '7941549408', 'orderId': 8864943117968240, 'productId': 237181694, 'productName': '샤바스 디자인이 예쁜 샤워타월 스퀘어', 'quantity': 1, 'gmv': 8370.0, 'commiss

In [476]:
len(df)

1620

In [473]:
df = pd.DataFrame(all_data)
len(df)

1620

In [422]:
api[api['orderDate']=="20240430"]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
64168,20240430,AF3155932,homecoin,73580666-071c-48b6-8a11-362b564eff8b_202404301...,460,7072823434,8684942817617311,7440025812,[리우] 레이스 브라 팬티 8종 세트,1,34800.0,5.25,1827.0
64169,20240430,AF3155932,homecoin,dc59d1ec-bc33-45ff-893e-ab6dfbac8216_202404291...,400,7978676284,8364941617637919,7183750807,매일바이오 드링킹 플레인 요구르트,3,9400.0,5.25,494.0
64170,20240430,AF3155932,homecoin,3c88cfba-5fd2-48c4-9de5-ef1dafc978b0_202404291...,460,7941549408,8864943117968240,237181694,샤바스 디자인이 예쁜 샤워타월 스퀘어,1,8370.0,5.25,440.0
64171,20240430,AF3155932,homecoin,bb1cff70-85c9-45e6-ac6a-8871957ed726,460,6497400457,8644944417532703,200355420,퓨어스펙 고당도 오렌지 중대과,1,6760.0,3.00,203.0
64172,20240430,AF3155932,homecoin,c785eab8-8e4c-4ab2-9e94-466e4d94f48d_202404302...,460,7737820744,8944940617198644,186181028,DECLU 케이블데코 플러스 2M 디자인 전선몰딩 풀패키지 세트 스몰사이즈,2,18420.0,5.25,968.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65783,20240430,AF3155932,homecoin,dc6caf11-0dea-4387-9784-c77957371c25_202404292...,400,5144429741,8334947917428229,7947437555,[3+1][당일출고] 페트리코어 링클프리 크롭 무지 기본티 여성 반팔티셔츠,1,29800.0,5.25,1565.0
65784,20240430,AF3155932,homecoin,6d33d297-840f-4731-90e1-92bfb99ef6f7_202404301...,460,7989645039,8674940974262862,2888447,잘풀리는집 리얼 실키 미용티슈 2겹,1,8880.0,5.25,467.0
65785,20240430,AF3155932,homecoin,594c1a44-7052-4dc6-bc81-eb50f7eaed80_202404292...,460,5233484318,8314942417712307,7154487253,피어스티나 피어싱 이어 홀 클리너 세트,1,9900.0,5.25,520.0
65786,20240430,AF3155932,homecoin,0879f515-a057-4604-b5f4-a98b2c2bd579_202404301...,460,7681939041,8684942717867013,7480077632,한국고려홍삼 뉴 홍삼정 굿데이즈 15gX60포+쇼핑백증정/6년근 홍삼,1,34900.0,5.25,1833.0


In [336]:
cancel = pd.read_parquet('../result/0403-0503_cp_cancelsReport.parquet')
order = pd.read_parquet('../result/0403-0501_cp_orderReport.parquet')

In [240]:
cancel = cancel.query("orderDate > '20240402' and orderDate < '20240502' ")
cancel.orderDate.min() , cancel.orderDate.max()


('20240403', '20240501')

## 취소건이랑 매칭 안되는 건 찾기

In [317]:

cancel.rename(columns={'quantity':'cancel_quantity', 'gmv':'cancel_gmv','commission':'cancel_commission'},inplace=True)
api.rename(columns={'date':'orderDate'},inplace=True)
cancel
mgd = pd.merge(cancel,api, on=['orderDate','trackingCode','subId','subParam','addtag','ctag','orderId'], how='left', indicator=True)


In [318]:
print(len(mgd[mgd['_merge']=='left_only']))
mgd[mgd['_merge']=='left_only'].tail()

10


,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId_x,productName_x,cancel_quantity,cancel_gmv,commissionRate_x,cancel_commission,productId_y,productName_y,quantity,gmv,commissionRate_y,commission,_merge
1068,20240403,20240409,AF3155932,homecoin,2f0cfc25-d145-4cdd-853d-6167c1f1af55,460,94168449,8324948815471576,6641041841,신지모루 자가복원 사생활보호 AG코팅 TPU 휴대폰 액정보호필름 2p 세트,-1,-13150.0,5.25,-691.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2253,20240403,20240411,AF3155932,homecoin,4d9d10ca-b0cd-4ac7-b720-186a5cb10735,460,6055649304,8094949515609000,6586217237,클레이탄 다이애나 로즈 원형 플레이트 S,-1,-25290.0,5.25,-1328.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2892,20240403,20240412,AF3155932,homecoin,db35d4b8-d38b-442a-bfbd-cb11f3a6c4a9,460,7881233348,8004943115858288,7899896222,데일리브 여성 사방스판 하이웨스트 히든밴딩 세미 일자핏 슬랙스 팬츠,-1,-25800.0,5.25,-1355.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2958,20240403,20240412,AF3155932,homecoin,92fbe121-e0a6-4699-81c7-cdcadc0436c4,400,7649949422,8604948515108218,7607637349,제로스트릿 소매 셔링 카라 블라우스,-1,-45000.0,5.25,-2363.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
7437,20240403,20240419,AF3155932,homecoin,,460,4600512109,8914945915979598,7790058218,여름 반팔 트레이닝복 세트 레터링 루즈핏 후드티 밴딩 조거팬츠 빅사이즈,-1,-21660.0,5.25,-1138.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only


### 상품명이 변경되는 케이스가 데이터 클렌징을 방해함

In [169]:
len(df[df['orderDate']<"20240502"]), len(cancel[cancel['orderDate']<"20240502"])
cancel[cancel['orderId']==8644944215953394]

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,cancel_quantity,cancel_gmv,commissionRate,cancel_commission
15,20240404,20240406,AF3155932,homecoin,1b299a08-c323-4e49-b59c-ec5014a2f6d3,400,7753775586,8964949215737481,7312573904,김영주 여성 세미 나팔 팬츠,-1,-49000.0,5.25,-2573.0


In [170]:
api[api['orderId']==8964949215737481]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
2710,20240404,AF3155932,homecoin,1b299a08-c323-4e49-b59c-ec5014a2f6d3,400,7753775586,8964949215737481,7312573904,김영주 여성 봄 여름 세미 나팔 팬츠 / 여자 나일론 스판 생활발수 골프웨어 아웃도...,1,49000.0,5.25,2573.0


### 상품아이디가 변경되는 케이스가 데이터 클렌징을 방해함

In [200]:
cancel[cancel['orderId']==8864943316144724]

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,cancel_quantity,cancel_gmv,commissionRate,cancel_commission
1471,20240411,20240412,AF3155932,homecoin,00558564-50a3-4eb6-aaa2-18cc2ea50b27,460,7320739567,8864943316144724,8026199285,다누리 뽀로로 복숭아맛 유아 비타민C 60g,-3,-11160.0,5.25,-586.0


In [201]:
api[api['orderId']==8864943316144724]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
25081,20240411,AF3155932,homecoin,00558564-50a3-4eb6-aaa2-18cc2ea50b27,460,7320739567,8864943316144724,5592286985,뽀로로 복숭아맛 유아 비타민C,3,11160.0,5.25,586.0


In [230]:
api[api['orderId']==8244947215821897]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
6553,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7558418551,[로켓프레시] 덴마크 동원 리코타 치즈,2,8920.0,5.25,469.0
6555,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,172859233,남양유업 맛있는우유GT 소화가 잘되는 고소한 락토프리,1,5980.0,5.25,314.0
6559,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7224566896,[로켓프레시] 그릭슈바인 육즙 가득 부어스트 스모크 소시지,1,6720.0,5.25,353.0
6574,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7243716923,[로켓프레시] 다향 훈제오리,1,15900.0,5.25,835.0
7414,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7455432237,[로켓프레시] 피그인더가든 데일리믹스 베이직샐러드,2,7680.0,5.25,404.0
7420,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7248140791,시티델리 칼조네 콰트로 포르마지 8개입 (냉동),1,7980.0,5.25,419.0
8233,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7248140790,시티델리 칼조네 콤비네이션 8개입 (냉동),1,7980.0,5.25,419.0
8234,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7487758475,[로켓프레시] 피그인더가든 프레쉬믹스 베이직샐러드,2,7680.0,5.25,404.0
8312,20240405,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,27925883,굽네 훈제 닭가슴살 오리지널,1,20520.0,5.25,1078.0


In [231]:
cancel[cancel['orderId']==8244947215821897]

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,cancel_quantity,cancel_gmv,commissionRate,cancel_commission
11,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7455432237,[로켓프레시] 피그인더가든 데일리믹스 베이직샐러드,-2,-7680.0,5.25,-404.0
12,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7248140790,시티델리 칼조네 콤비네이션 8개입 (냉동),-1,-7980.0,5.25,-419.0
13,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7243716923,[로켓프레시] 다향 훈제오리,-1,-15900.0,5.25,-835.0
14,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,172859233,남양유업 맛있는우유GT 소화가 잘되는 고소한 락토프리,-1,-5980.0,5.25,-314.0


In [236]:
cancel[cancel['orderId']==8244947215821897]

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,cancel_quantity,cancel_gmv,commissionRate,cancel_commission
11,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7455432237,[로켓프레시] 피그인더가든 데일리믹스 베이직샐러드,-2,-7680.0,5.25,-404.0
12,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7248140790,시티델리 칼조네 콤비네이션 8개입 (냉동),-1,-7980.0,5.25,-419.0
13,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7243716923,[로켓프레시] 다향 훈제오리,-1,-15900.0,5.25,-835.0
14,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,172859233,남양유업 맛있는우유GT 소화가 잘되는 고소한 락토프리,-1,-5980.0,5.25,-314.0


In [241]:
cancel.head(1)

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
0,20240403,20240406,AF3155932,homecoin,,460,6970140835,8054946215009009,6751133973,오렌지다몸 여아용 XX 스마일 맨투맨 + 레깅스 세트,-1,-23500.0,5.25,-1234.0


### 최선은 productId 까지 포함한 뒤, 나머지는 productId가 변경된 것으로 간주하고 후처리

In [368]:
cancel.sort_values(by='orderDate')

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
637,20240115,20240409,AF3155932,homecoin,,460,3325924,8864943808461811,162144036,클래파 스틱 & 핸디 2in1 고효율 ERP헤드 싸이클론 유선청소기 IB-VC0601,-1,-22880.0,5.50,-1259.0
879,20240224,20240410,AF3155932,homecoin,,400,322819155,8874947827304465,10000010148820,[대구] 네이처파크 스윗윈터페스티벌 1+1(2인권) 특가#_D:(WOW)종일권 1+...,-2,-19800.0,5.25,-1040.0
1025,20240308,20240411,AF3155932,homecoin,,460,6444223952,8694941189540462,10000010166201,[전국] 도심형 실내 키즈 테마파크 챔피언/1250/블랙벨트 57개점 특가#_D:경...,-2,-45800.0,5.25,-2405.0
615,20240309,20240409,AF3155932,homecoin,,400,7658152716,8824945789947529,7400212905,미주라 통밀크래커,-1,-13420.0,5.25,-705.0
1503,20240314,20240413,AF3155932,homecoin,,400,7472931059,8314942180716988,6339684425,회전식 수납 화장품 양념장 주방 화장대 선반,-1,-6680.0,5.25,-351.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5073,20240503,20240503,AF3155932,homecoin,4823e03d-a879-4780-aa52-07904e57bd50_202405021...,460,7774680738,8004943874831144,7490966106,홈플래닛 C타입 레인보우 고속 충전 케이블 (삼성 갤럭시 애플 아이폰15 호환),-1,-9990.0,5.25,-525.0
5075,20240503,20240503,AF3155932,homecoin,482cfcca-6914-49bd-a2d7-f7d05f6269c5_202405021...,460,94167476,8324948175685552,7844569752,[우연팜] 슈퍼푸드 제주 콜라비 1kg 2kg 3kg 5kg 10kg 무료배송,-1,-8810.0,5.25,-463.0
5115,20240503,20240503,AF3155932,homecoin,6aaf12bc-cc5d-4115-aeef-9cf42aaba510_202405011...,460,6622899113,8824945274577058,2315399066,오리온 꼬북칩 초코츄러스 맛,-3,-5370.0,5.25,-282.0
5051,20240503,20240503,AF3155932,homecoin,1f901a60-2f78-41f8-b03d-1af6a13212ac_202405021...,460,7260696703,8684942974511974,7198631640,종근당건강 락토핏 슬림 유산균 60p,-1,-18060.0,5.25,-949.0


In [369]:

# cancel.rename(columns={'quantity':'cancel_quantity', 'gmv':'cancel_gmv','commission':'cancel_commission'},inplace=True)
api.rename(columns={'date':'orderDate'},inplace=True)

mgd = pd.merge(cancel,api, on=['orderDate','trackingCode','subId','addtag','ctag','orderId','productId'], how='left', indicator=True)


In [385]:
# 취소 테이블에 있는 주문 이력이 주문 테이블에는 존재하지 않음.
api[api['orderId']==8314942115880569]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission


In [374]:
#취소 테이블과 주문 테이블 중 매칭에 실패하고 남은 잔여 취소건
print('나머지' , len(mgd[mgd['_merge']=='left_only']))
mgd[(mgd['_merge']=='left_only') & (mgd['orderDate']=='20240403')]


나머지 352


,orderDate,date,trackingCode,subId,subParam_x,addtag,ctag,orderId,productId,productName_x,quantity_x,gmv_x,commissionRate_x,commission_x,subParam_y,productName_y,quantity_y,gmv_y,commissionRate_y,commission_y,_merge
2,20240403,20240406,AF3155932,homecoin,,400,7409691034,8644944215953394,70018077,맘스덤 하이드로 밴드,-1,-4170.0,5.25,-219.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
112,20240403,20240407,AF3155932,homecoin,dc5de31f-57fc-4f1e-935b-c3a33364ed72,460,7052231187,8054946615130902,7578284121,꼰띠키즈 아동용 토로토 상하복 세트,-1,-23800.0,5.25,-1250.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
164,20240403,20240407,AF3155932,homecoin,,460,7958337612,8874947415049614,6518007021,데이프릴 여성용 크론 하이플랫폼 6cm 키높이 샌들 슬리퍼 아쿠아슈즈,-1,-14800.0,5.25,-777.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
308,20240403,20240408,AF3155932,homecoin,f3c10b7f-8502-4a67-9151-334cb2791187,460,7920195446,8314942115880569,7116582265,ELLE PARIS 링클프리 루즈핏 롱 윈드점퍼,-1,-31490.0,5.25,-1654.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
355,20240403,20240408,AF3155932,homecoin,87da0196-720b-4d1e-8d28-d4b59efd4752,460,7215277043,8914945115060280,7988633344,나이키 NSW 클럽 프렌치테리 조거팬츠,-1,-29900.0,5.25,-1570.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
374,20240403,20240408,AF3155932,homecoin,f3c10b7f-8502-4a67-9151-334cb2791187,460,7920195446,8314942115980074,7839394078,ELLEPARIS 스트링 후드 바람막이 점퍼,-1,-27280.0,5.25,-1433.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
444,20240403,20240408,AF3155932,homecoin,,460,6792703955,8054946515646063,7939574059,봄 여름 얇은 긴팔 플라워 원피스,-1,-23900.0,5.25,-1255.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
503,20240403,20240409,AF3155932,homecoin,2f0cfc25-d145-4cdd-853d-6167c1f1af55,460,94168449,8324948815471576,6641041841,신지모루 자가복원 사생활보호 AG코팅 TPU 휴대폰 액정보호필름 2p 세트,-1,-13150.0,5.25,-691.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1006,20240403,20240411,AF3155932,homecoin,4d9d10ca-b0cd-4ac7-b720-186a5cb10735,460,6055649304,8094949515609000,6586217237,클레이탄 다이애나 로즈 원형 플레이트 S,-1,-25290.0,5.25,-1328.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1221,20240403,20240412,AF3155932,homecoin,db35d4b8-d38b-442a-bfbd-cb11f3a6c4a9,460,7881233348,8004943115858288,7899896222,데일리브 여성 사방스판 하이웨스트 히든밴딩 세미 일자핏 슬랙스 팬츠,-1,-25800.0,5.25,-1355.0,NaN,NaN,NaN,NaN,NaN,NaN,left_only


이런 케이스는 뭘까.... 주문, 취소 데이터에는 분명히 있는데 UserId, orderId가 변했다..

In [294]:
cancel[cancel['orderId']==8314942115880569]

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
308,20240403,20240408,AF3155932,homecoin,f3c10b7f-8502-4a67-9151-334cb2791187,460,7920195446,8314942115880569,7116582265,ELLE PARIS 링클프리 루즈핏 롱 윈드점퍼,-1,-31490.0,5.25,-1654.0


In [295]:
api[api['orderId']==8314942115880569]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission


In [300]:
api[api['productId']==7116582265]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
43719,20240418,AF3155932,homecoin,51eaddb9-5547-4d2e-8dc3-7d181ea2f77e,460,7284994813,8834949812165560,7116582265,ELLE PARIS 링클프리 루즈핏 롱 윈드점퍼,1,31490.0,5.25,1654.0


In [298]:
online_order[online_order['상품 ID']==7116582265]

,날짜,채널 ID,상품 ID,상품명,판매 수량,매출 액,수익,취소 수량,취소 액,취소 수익
1891,2024-04-03,homecoin,7116582265,ELLE PARIS 링클프리 루즈핏 롱 윈드점퍼,1,31490,1654,-1,-31490,-1654
14881,2024-04-08,homecoin,7116582265,ELLE PARIS 링클프리 루즈핏 롱 윈드점퍼,0,0,0,-1,-31490,-1654
41079,2024-04-18,homecoin,7116582265,ELLE PARIS 링클프리 루즈핏 롱 윈드점퍼,1,31490,1654,0,0,0


In [309]:

api[api['productId']==96849	]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
1303,20240403,AF3155932,homecoin,2d35cae9-d7ad-4022-b249-0fe9e32de756,460,7692707795,8074944715588594,96849,빙그레 뽀로로와친구들 플레인 크래커,3,3750.0,5.25,197.0
1799,20240403,AF3155932,homecoin,2d35cae9-d7ad-4022-b249-0fe9e32de756,460,7692707795,8074944115268519,96849,빙그레 뽀로로와친구들 플레인 크래커,3,3750.0,5.25,197.0
32501,20240414,AF3155932,homecoin,623aeecb-f74b-47b1-984f-cd19b2d9eeaf,460,7623789196,8824945716327345,96849,빙그레 뽀로로와친구들 플레인 크래커,6,7200.0,5.25,378.0


In [316]:
cancel[cancel['orderId']==8824945716327345]

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission


In [390]:
api.gmv.sum() + cancel.query("orderDate < '20240502'").gmv.sum()

1021978478.0

리포트 데이터도 개이사함

In [324]:
online_order[(online_order['판매 수량']!=0) & (online_order['취소 수량']==-1)]
online_order[online_order['상품 ID']==1835151]


,날짜,채널 ID,상품 ID,상품명,판매 수량,매출 액,수익,취소 수량,취소 액,취소 수익,합산거래액
60,2024-04-03,homecoin,1835151,스파이더락 와이어 250 선반,0,0,0,-1,-11680,-614,NaN


In [356]:
df[df['productId']==61024]

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission


In [359]:
df.date.min(), df.date.max()
df.orderDate.min(), df.orderDate.max()

('20231201', '20240331')

In [361]:
df.sort_values(by='orderDate')

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission
2674,20231201,20240328,AF3155932,homecoin,,460,7632949814,8314942166287905,7295558262,로지텍 코리아 리프트 버티컬 인체공학 무선 블루투스 마우스 LIFT VERTICAL...,-1,-76180.0,5.50,-4190.0
1965,20240103,20240325,AF3155932,homecoin,,400,6717005203,8054946206640998,310110943,엔뚜마노 보니또 전기포트 1.7L,-1,-35910.0,5.50,-1976.0
1239,20240111,20240321,AF3155932,homecoin,,400,6518120220,8654947402182477,7699125817,포니츠 포버튼 하이웨스트 스키니진,-2,-59800.0,5.50,-3290.0
273,20240214,20240316,AF3155932,homecoin,,460,7258087545,8844946122373044,1475389,엠디프라임 황토 냉온 찜질팩,-1,-4920.0,5.25,-259.0
517,20240216,20240318,AF3155932,homecoin,,460,7544102564,8004943028555454,7687816904,로지텍 무선키보드,-1,-39950.0,5.25,-2098.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3233,20240331,20240331,AF3155932,homecoin,,460,5815712263,8084945714429360,6232629971,고려 흑홍삼캔디 어르신건강 간식 270g 5봉지,-1,-29500.0,5.25,-1549.0
3143,20240331,20240331,AF3155932,homecoin,,460,7049969064,8004943614412775,553986,케라시스 프로폴리스 샤인 트리트먼트,-1,-6330.0,5.25,-333.0
3236,20240331,20240331,AF3155932,homecoin,,460,7539694494,8214940514926535,7055258864,미디어 2 in 1 유선 진공 청소기,-1,-20900.0,5.25,-1098.0
3238,20240331,20240331,AF3155932,homecoin,,460,4710736457,8244947714474614,7721321525,김수열줄넘기 스마트 명품 줄넘기 SY-100,-1,-6690.0,5.25,-352.0


In [322]:
online_order['합산거래액'] = online_order[online_order['판매 수량'] > 0]['매출 액'] + online_order[online_order['판매 수량'] > 0]['취소 액']
online_order['합산거래액'].sum()

1087879392.0

In [307]:
online_order[(online_order['판매 수량']!=0) & (online_order['취소 수량']<=-1)]

,날짜,채널 ID,상품 ID,상품명,판매 수량,매출 액,수익,취소 수량,취소 액,취소 수익
2,2024-04-03,homecoin,14567,잘풀리는집 천연펄프 키친타올 130매,3,14490,763,-1,-4880,-257
23,2024-04-03,homecoin,96849,빙그레 뽀로로와친구들 플레인 크래커,6,7500,394,-3,-3750,-197
46,2024-04-03,homecoin,1004408,마그피아 플라스틱 칼라링 200개입,1,4450,234,-1,-4450,-234
82,2024-04-03,homecoin,2598181,하리보 골드베렌젤리,2,25280,1328,-1,-12640,-664
92,2024-04-03,homecoin,2990719,레인보우샵 베이킹소다 베이직,6,15780,832,-1,-2630,-139
...,...,...,...,...,...,...,...,...,...,...
62474,2024-05-01,homecoin,7949370857,지홍 JIHONG 여름 반팔 프릴 원피스 루즈핏 셔츠원피스,1,43900,2305,-1,-43900,-2305
62508,2024-05-01,homecoin,7983964103,햇살닭 더촉촉한 닭가슴살 외 12종 12팩(각 1팩씩) / 소스 / 혼합구성,1,18800,987,-1,-18800,-987
62542,2024-05-01,homecoin,8026852965,질레트 스킨텍 면도날 10p + 면도기 스탠드 세트,1,58750,3085,-1,-58750,-3085
62549,2024-05-01,homecoin,8037319915,라이프홀릭 퀵 라이트 쿨링 롤온 데오드란트,1,5010,264,-1,-5010,-264


In [304]:
online_order[online_order['상품 ID']==37366741]

,날짜,채널 ID,상품 ID,상품명,판매 수량,매출 액,수익,취소 수량,취소 액,취소 수익
177,2024-04-03,homecoin,37366741,조아캠프 캠핑테이블 120 + 전용가방,0,0,0,-1,-25060,-1316


다시 취소건 처리하여 엑셀 시트와 매출을 비교하자

In [221]:
cancel.rename(columns={'quantity':'cancel_quantity', 'gmv':'cancel_gmv','commission':'cancel_commission'},inplace=True)
api.rename(columns={'date':'orderDate'},inplace=True)
cancel
mgd = pd.merge(api,cancel, on=['orderDate','trackingCode','subId','subParam','addtag','ctag','orderId'], how='left')
mgd

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId_x,productName_x,quantity,gmv,commissionRate_x,commission,date,productId_y,productName_y,cancel_quantity,cancel_gmv,commissionRate_y,cancel_commission
0,20240403,AF3155932,homecoin,,460,1396937977,8214940115445302,7842640108,프라임큐 삼성 갤럭시 호환 C타입 유선 오픈형 이어폰,1,4060.0,5.25,214.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240403,AF3155932,homecoin,,460,6055129082,8964949215036459,1258945808,곰곰 소중한 우리쌀 밥,1,28990.0,5.25,1522.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20240403,AF3155932,homecoin,,460,7953413848,8264948615253834,1649813767,풀무원 무농약 인증 국산 숙주,1,1990.0,5.25,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20240403,AF3155932,homecoin,,460,7580419077,8694941415136593,7217502946,태양소방 3.3kg ABC 분말소화기,1,18490.0,5.25,971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20240403,AF3155932,homecoin,,460,7771483157,8964949515710586,7748460732,서울우유 나100%우유,1,5340.0,5.25,281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74194,20240501,AF3155932,homecoin,78f3e5ed-42d3-453f-a6dc-6297e9fb23aa_202405011...,460,5638415311,8694941474529259,7154237009,나비다 여성용 크롭 니트 반팔 티셔츠,1,13900.0,5.25,730.0,20240503,7.154237e+09,나비다 여성용 크롭 니트 반팔 티셔츠,-1.0,-13900.0,5.25,-730.0
74195,20240501,AF3155932,homecoin,e0638bb0-4563-4974-bf1b-3f5c27ab1c22_202405011...,460,6175056549,8814944174993861,55502828,쁘띠누베 실리콘 이유식스푼 2p + 케이스,1,6980.0,5.25,367.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74196,20240501,AF3155932,homecoin,0139ea74-6386-4b9f-8c22-7f629b5b1aa1_202405011...,400,1796950494,8004943574656184,7912528838,자연을 담은 포레스트 플랜테리어 패브릭포스터 50종 가리개 커튼 인테리어 소품 대형...,1,9800.0,5.25,515.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74197,20240501,AF3155932,homecoin,52a41f08-f9d3-4881-990e-51faed974fd1_202405012...,460,7634497187,8654947974825001,6942908972,아디다스 CORE RC 남성 에브리데이 런닝화 블랙화이트 PG3903,1,42120.0,5.25,2212.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
print("API 중복 행 수:", api.duplicated(subset=['orderDate', 'trackingCode', 'subId', 'subParam', 'addtag', 'ctag', 'orderId','productName']).sum())
print("Cancel 중복 행 수:", cancel.duplicated(subset=['orderDate', 'trackingCode', 'subId', 'subParam', 'addtag', 'ctag', 'orderId']).sum())

API 중복 행 수: 18
Cancel 중복 행 수: 1151


In [227]:
api[api.duplicated(subset=['orderDate', 'trackingCode', 'subId', 'subParam', 'addtag', 'ctag', 'orderId','productId' ,'productName'])]

,orderDate,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,quantity,gmv,commissionRate,commission


In [229]:
cancel[cancel.duplicated(subset=['orderDate', 'trackingCode', 'subId', 'subParam', 'addtag', 'ctag', 'orderId'])]

,orderDate,date,trackingCode,subId,subParam,addtag,ctag,orderId,productId,productName,cancel_quantity,cancel_gmv,commissionRate,cancel_commission
1,20240403,20240406,AF3155932,homecoin,,460,6970140835,8054946215009009,7048305528,베키쥬 월터 기본핏 라인 상하 세트,-1,-15900.0,5.25,-835.0
12,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7248140790,시티델리 칼조네 콤비네이션 8개입 (냉동),-1,-7980.0,5.25,-419.0
13,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,7243716923,[로켓프레시] 다향 훈제오리,-1,-15900.0,5.25,-835.0
14,20240405,20240406,AF3155932,homecoin,afe167c1-b3cc-4612-a494-6ddd9eb11fa9,460,195039890,8244947215821897,172859233,남양유업 맛있는우유GT 소화가 잘되는 고소한 락토프리,-1,-5980.0,5.25,-314.0
17,20240406,20240406,AF3155932,homecoin,53652fc4-79ba-4c9b-b9d9-26f286c1049f,460,4507437901,8934946913546376,1459744430,TSP 소프트 그립 양용 드라이버 150mm,-1,-4750.0,5.25,-250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5068,20240407,20240503,AF3155932,homecoin,bb8dd2f6-27c2-47b4-b302-6f615940c74f,460,2123688601,8684942813738662,7501048197,구디푸디 유아용 썸머패치 밴드 스티커 18p 세트,-1,-6610.0,5.25,-348.0
5080,20240430,20240503,AF3155932,homecoin,594c1a44-7052-4dc6-bc81-eb50f7eaed80_202404292...,460,5233484318,8314942417712307,7569701634,피어니스트 여성용 허리밴딩 카코 트레이닝 팬츠,-1,-24920.0,5.25,-1309.0
5161,20240425,20240503,AF3155932,homecoin,41b20ad4-83ee-4a2d-a270-234375eee132_202404241...,460,5221721596,8834949611744999,6870217849,필립스 SkinIQ 9000 프레스티지 시리즈 전기면도기,-2,-834560.0,5.25,-43815.0
5162,20240425,20240503,AF3155932,homecoin,41b20ad4-83ee-4a2d-a270-234375eee132_202404241...,460,5221721596,8834949611744999,2041166639,필립스 SkinIQ 7000 시리즈 전기면도기,-2,-377060.0,5.25,-19796.0


In [4]:
import openpyxl

In [11]:
online_order = pd.read_excel('../result/129603-daily-trend-AF3155932-2024-04-03-2024-05-01.xlsx',engine='openpyxl',sheet_name=1)
# api = pd.read_parquet('../result/0403-0501_cp_orderReport.parquet')

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [23]:
print('온라인_거래액 : ', online_order['매출 액'].sum(), "API_거래액 : ", api.gmv.sum(), "차이 : ", round(1- online_order['매출 액'].sum() / api.gmv.sum(),2))

온라인_거래액 :  1166290512 API_거래액 :  1149589196.01 차이 :  -0.01


In [56]:
print( online_order['취소 수량'].sum() /  online_order['판매 수량'].sum())
print(round(online_order[online_order['취소 액']==0]['매출 액'].sum() /   online_order[online_order['취소 액']<0]['매출 액'].sum(),2))

-0.07974716231236458
9.35


In [78]:
online_order['매출 액'].sum() + online_order['취소 액'].sum()

1020047124

In [43]:
online_order

,날짜,채널 ID,상품 ID,상품명,판매 수량,매출 액,수익,취소 수량,취소 액,취소 수익
0,2024-04-03,homecoin,10869,토퍼 테비 플러스케어 고양이 사료,1,17090,898,0,0,0
1,2024-04-03,homecoin,12547,네이쳐스웨이 크랜베리 비건 캡슐,4,65280,3428,0,0,0
2,2024-04-03,homecoin,14567,잘풀리는집 천연펄프 키친타올 130매,3,14490,763,-1,-4880,-257
3,2024-04-03,homecoin,23118,비트 드럼용 액상세제 리필,1,8957,471,0,0,0
4,2024-04-03,homecoin,29876,폰타나 엑스트라버진 올리브유,2,25260,1327,0,0,0
...,...,...,...,...,...,...,...,...,...,...
62554,2024-05-01,homecoin,8057098903,잇진 여성 하이틴 워싱 데님 미니 스커트 청치마,1,19800,1040,0,0,0
62555,2024-05-01,homecoin,8057226231,GRN 시즌4 올 뉴 초록이 다이어트 식품,2,31340,1646,0,0,0
62556,2024-05-01,homecoin,8062003947,닥터블릿 푸응 팻버닝 CLA 공액 리놀레산 18g,1,29400,1544,0,0,0
62557,2024-05-01,homecoin,8065752480,포카리스웨트 무라벨,1,14630,769,0,0,0


In [17]:
# api[api['productName']=='잘풀리는집 천연펄프 키친타올 130매']
api.gmv.sum()

1149589196.01